# Amplitude embedding (quantum hardware)

Main steps:

* initialize an image of the sky (frequency domain)
* apply 2D FT --> visibilities (Fourier domain)
* encode visibilities data into qubits using amplitude embedding (non-conventional domain)
* measure qubits on real quantum hardware --> expected outcomes (back to conventional domain)
* apply 2D IFT --> fidelity computation

In [53]:
import numpy as np
import math
import sys

import pennylane as qml
from pennylane import numpy as pnp

## 1) Generating an artificial image of the sky (frequency domain)

In [61]:
#image of the sky filled with single precision complex floats ('complex64')
#pixels are set to low complex random values (image background/noise) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype='complex64')
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[6.9207565e+01+3.9058460e+01j 6.9207565e+01+3.9058460e+01j
  7.1590641e-03+9.3000764e-03j 6.9207565e+01+3.9058460e+01j]
 [6.9207565e+01+3.9058460e+01j 2.2826253e-03+2.8941694e-03j
  9.5535852e-03+9.7155832e-03j 1.7066648e-03+7.1718167e-03j]
 [9.1994358e-03+7.4965730e-03j 8.1475247e-03+5.7593943e-03j
  8.1866635e-03+8.7047163e-03j 8.8655343e-03+5.7419911e-03j]
 [6.9207565e+01+3.9058460e+01j 2.4715678e-03+9.7065680e-03j
  6.0474677e-03+7.9706488e-03j 7.8072539e-03+6.8479711e-03j]]


## 2) Applying a 2D FT (Fourier domain)

In [62]:
visibilities = np.fft.fft2(sky_image).astype('complex64')
print(visibilities)

[[ 346.10925 +195.37361j   207.59955 +117.152664j   69.21643  +39.063526j
   207.60236 +117.14171j ]
 [ 207.59071 +117.15976j    69.19172  +39.06029j   -69.19254  -39.063656j
    69.203575 +39.047455j]
 [  69.21925  +39.051155j  -69.19667  -39.055325j -207.6165  -117.15244j
   -69.19955  -39.04724j ]
 [ 207.6002  +117.1542j     69.20703  +39.03901j   -69.209015 -39.044064j
    69.195244 +39.054718j]]


#### Sanity check

In [63]:
img = np.fft.ifft2(visibilities).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[6.9207565e+01+3.9058460e+01j 6.9207565e+01+3.9058460e+01j
  7.1587563e-03+9.3004704e-03j 6.9207565e+01+3.9058460e+01j]
 [6.9207565e+01+3.9058460e+01j 2.2823811e-03+2.8934479e-03j
  9.5536709e-03+9.7155571e-03j 1.7068386e-03+7.1721077e-03j]
 [9.1996193e-03+7.4965954e-03j 8.1472397e-03+5.7590008e-03j
  8.1849098e-03+8.7049007e-03j 8.8653564e-03+5.7418346e-03j]
 [6.9207565e+01+3.9058460e+01j 2.4712086e-03+9.7069740e-03j
  6.0474873e-03+7.9703331e-03j 7.8065395e-03+6.8492889e-03j]]

Reals MSE: 2.5334802e-13
Imaginaries MSE: 1.8601434e-13


## 3) Data encoding: amplitude embedding (non-conventional domain)

In [64]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
n = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(n))) # set the number of qbits (no padding needed if outputs an integer=integer.0)
amp_dev = qml.device('default.qubit', wires)

@qml.qnode(amp_dev)
def amp_encoding(data):
    qml.AmplitudeEmbedding(data, wires) # normalize = True
    return qml.state() #qml.expval(qml.PauliZ(wires=wires))

results = amp_encoding(normalized_data).astype('complex64')
print(results)

print()

results = results*qml.math.sqrt(norm) # denormalization of the measurements outcomes
results = np.array(results).reshape(sky_image.shape[0] , sky_image.shape[1])
print(results)

[ 0.48693717+0.27486891j  0.29206944+0.16482075j  0.0973798 +0.05495803j
  0.2920734 +0.16480534j  0.29205701+0.16483073j  0.09734504+0.05495348j
 -0.0973462 -0.05495821j  0.09736172+0.05493542j  0.09738377+0.05494063j
 -0.09735201-0.05494649j -0.29209328-0.16482043j -0.09735605-0.05493512j
  0.29207036+0.16482291j  0.09736659+0.05492354j -0.09736937-0.05493065j
  0.09735   +0.05494564j]

[[ 346.10925 +195.37361j   207.59955 +117.152664j   69.21643  +39.063526j
   207.60236 +117.14171j ]
 [ 207.59071 +117.15976j    69.19172  +39.06029j   -69.19254  -39.063656j
    69.203575 +39.047455j]
 [  69.21925  +39.051155j  -69.19667  -39.055325j -207.61649 -117.15244j
   -69.19955  -39.04724j ]
 [ 207.6002  +117.1542j     69.20703  +39.03901j   -69.209015 -39.044064j
    69.195244 +39.054718j]]


## 4) Applying 2D IFT (fidelity test)

In [65]:
img = np.fft.ifft2(results).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[6.9207565e+01+3.9058460e+01j 6.9207565e+01+3.9058460e+01j
  7.1597099e-03+9.3004704e-03j 6.9207565e+01+3.9058460e+01j]
 [6.9207565e+01+3.9058460e+01j 2.2833347e-03+2.8934479e-03j
  9.5527172e-03+9.7155571e-03j 1.7077923e-03+7.1721077e-03j]
 [9.2005730e-03+7.4965954e-03j 8.1462860e-03+5.7590008e-03j
  8.1858635e-03+8.7049007e-03j 8.8644028e-03+5.7418346e-03j]
 [6.9207565e+01+3.9058460e+01j 2.4721622e-03+9.7069740e-03j
  6.0465336e-03+7.9703331e-03j 7.8074932e-03+6.8492889e-03j]]

Reals MSE: 5.610047e-13
Imaginaries MSE: 1.8601434e-13


## Utils

### Quantum

### Classical

In [60]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask